In [1]:
FN = 'predict'

In [2]:
import os

In [3]:
import keras
keras.__version__


Using Theano backend.


'2.2.4'

In [4]:
FN0 = 'vocabulary-embedding'

In [5]:
FN1 = 'train'

In [6]:
maxlend=50 # 0 - if we dont want to use description at all
maxlenh=25
maxlen = maxlend + maxlenh
rnn_size = 512
rnn_layers = 3  # match FN1
batch_norm=False


In [7]:
activation_rnn_size = 40 if maxlend else 0

In [8]:
seed=42
p_W, p_U, p_dense, p_emb, weight_decay = 0, 0, 0, 0, 0
optimizer = 'adam'
batch_size=64

In [9]:
nb_train_samples = 30000
nb_val_samples = 3000

In [10]:
import pickle

with open('%s.pkl'%FN0, 'rb') as fp:
    embedding, idx2word, word2idx, glove_idx2idx = pickle.load(fp)
vocab_size, embedding_size = embedding.shape


In [11]:
nb_unknown_words = 10

In [59]:
print('dimension of embedding space for words',embedding_size)
print(vocab_size)
print('vocabulary size', vocab_size, 'the last %d words can be used as place holders for unknown/oov words'%nb_unknown_words)
print('total number of different words',len(idx2word), len(word2idx))
print('number of words outside vocabulary which we can substitue using glove similarity', len(glove_idx2idx))
print('number of words that will be regarded as unknonw(unk)/out-of-vocabulary(oov)',len(idx2word)-vocab_size-len(glove_idx2idx))

dimension of embedding space for words 100
40000
vocabulary size 40000 the last 10 words can be used as place holders for unknown/oov words
total number of different words 6723718 6723718
number of words outside vocabulary which we can substitue using glove similarity 241023
number of words that will be regarded as unknonw(unk)/out-of-vocabulary(oov) 6442695


In [13]:
for i in range(nb_unknown_words):
    idx2word[vocab_size-1-i] = '<%d>'%i

In [14]:
for i in range(vocab_size-nb_unknown_words, len(idx2word)):
    idx2word[i] = idx2word[i]+'^'

In [15]:
empty = 0
eos = 1
idx2word[empty] = '_'
idx2word[eos] = '~'

In [16]:
import numpy as np
from keras.preprocessing import sequence
from keras.utils import np_utils
import random, sys


In [17]:
def prt(label, x):
    print(label+':')
    for w in x:
        print(idx2word[w])
    print()

In [18]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout, RepeatVector
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.regularizers import l2
from keras.layers.core import Lambda
import keras.backend as K

In [19]:
random.seed(seed)
np.random.seed(seed)


In [20]:
regularizer = l2(weight_decay) if weight_decay else None

In [21]:
rnn_model = Sequential()
rnn_model.add(Embedding(vocab_size, embedding_size,
                        input_length=maxlen,
                        W_regularizer=regularizer, dropout=p_emb, weights=[embedding], mask_zero=True,
                        name='embedding_1'))
for i in range(rnn_layers):
    lstm = LSTM(rnn_size, return_sequences=True, # batch_norm=batch_norm,
                W_regularizer=regularizer, U_regularizer=regularizer,
                b_regularizer=regularizer, dropout_W=p_W, dropout_U=p_U,
                name='lstm_%d'%(i+1)
                  )
    rnn_model.add(lstm)
    rnn_model.add(Dropout(p_dense, name='dropout_%d'%(i+1)))

/home/yashomdighe/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  """
/home/yashomdighe/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(40000, 100, input_length=75, weights=[array([[-..., mask_zero=True, name="embedding_1", embeddings_regularizer=None)`
  """
/home/yashomdighe/.local/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(512, return_sequences=True, name="lstm_1", kernel_regularizer=None, bias_regularizer=None, recurrent_regularizer=None, dropout=0, recurrent_dropout=0)`
  # Remove the CWD from sys.path while we load stuff.
/home/yashomdighe/.local/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `L

In [22]:
import h5py
def str_shape(x):
    return 'x'.join(map(str,x.shape))

def inspect_model(model):
    print(model.name)
    for i,l in enumerate(model.layers):
        print(i, 'cls=%s name=%s'%(type(l).__name__, l.name))
        weights = l.get_weights()
        for weight in weights:
            print(str_shape(weight))
        print()

In [23]:
def load_weights(model, filepath):
    """Modified version of keras load_weights that loads as much as it can
    if there is a mismatch between file and model. It returns the weights
    of the first layer in which the mismatch has happened
    """
    print('Loading', filepath, 'to', model.name)
    flattened_layers = model.layers
    with h5py.File(filepath, mode='r') as f:
        # new file format
        layer_names = [n.decode('utf8') for n in f.attrs['layer_names']]

        # we batch weight value assignments in a single backend call
        # which provides a speedup in TensorFlow.
        weight_value_tuples = []
        for name in layer_names:
            print(name)
            g = f[name]
            weight_names = [n.decode('utf8') for n in g.attrs['weight_names']]
            if len(weight_names):
                weight_values = [g[weight_name] for weight_name in weight_names]
                try:
                    layer = model.get_layer(name=name)
                except:
                    layer = None
                if not layer:
                    print('failed to find layer', name, 'in model')
                    print('weights', ' '.join(str_shape(w) for w in weight_values))
                    print('stopping to load all other layers')
                    weight_values = [np.array(w) for w in weight_values]
                    break
                symbolic_weights = layer.trainable_weights + layer.non_trainable_weights
                weight_value_tuples += zip(symbolic_weights, weight_values)
                weight_values = None
        K.batch_set_value(weight_value_tuples)
    return weight_values

In [24]:
weights = load_weights(rnn_model, '%s.hdf5'%FN1)

Loading train.hdf5 to sequential_1
embedding_1
lstm_1
dropout_1
lstm_2
dropout_2
lstm_3
dropout_3
simplecontext_1
time_distributed_1
failed to find layer time_distributed_1 in model
weights 944x40000 40000
stopping to load all other layers


In [25]:
[w.shape for w in weights]

[(944, 40000), (40000,)]

In [26]:
context_weight = K.variable(1.)
head_weight = K.variable(1.)
cross_weight = K.variable(0.)

In [29]:
def simple_context(X, mask, n=activation_rnn_size, maxlend=maxlend, maxlenh=maxlenh):
    desc, head = X[:,:maxlend], X[:,maxlend:]
    head_activations, head_words = head[:,:,:n], head[:,:,n:]
    desc_activations, desc_words = desc[:,:,:n], desc[:,:,n:]
    
    # RTFM http://deeplearning.net/software/theano/library/tensor/basic.html#theano.tensor.batched_tensordot
    # activation for every head word and every desc word
    activation_energies = K.batch_dot(head_activations, desc_activations, axes=(2,2))
    # make sure we dont use description words that are masked out
    assert mask.ndim == 2
    activation_energies = K.switch(mask[:, None, :maxlend], activation_energies, -1e20)
    
    # for every head word compute weights for every desc word
    activation_energies = K.reshape(activation_energies,(-1,maxlend))
    activation_weights = K.softmax(activation_energies)
    activation_weights = K.reshape(activation_weights,(-1,maxlenh,maxlend))

    # for every head word compute weighted average of desc words
    desc_avg_word = K.batch_dot(activation_weights, desc_words, axes=(2,1))
    return K.concatenate((context_weight*desc_avg_word, head_weight*head_words))


class SimpleContext(Lambda):
    def __init__(self,**kwargs):
        super(SimpleContext, self).__init__(simple_context,**kwargs)
        self.supports_masking = True

    def compute_mask(self, input, input_mask=None):
        print(input_mask[:, maxlend:])
        return input_mask[:, maxlend:]
    
    def compute_output_shape(self, input_shape):
        nb_samples = input_shape[0]
        n = 2*(rnn_size - activation_rnn_size)
        print((nb_samples, maxlenh, n))
        return (nb_samples, maxlenh, n)


In [30]:
model = Sequential()
model.add(rnn_model)

if activation_rnn_size:
    model.add(SimpleContext(name='simplecontext_1'))

Subtensor{::, int64::}.0
(None, 25, 944)


In [31]:
# we are not going to fit so we dont care about loss and optimizer
model.compile(loss='categorical_crossentropy', optimizer='adam')


In [32]:
n = 2*(rnn_size - activation_rnn_size)
n

944

In [33]:
# out very own softmax
def output2probs(output):
    output = np.dot(output, weights[0]) + weights[1]
    output -= output.max()
    output = np.exp(output)
    output /= output.sum()
    return output

In [34]:
def output2probs1(output):
    output0 = np.dot(output[:n//2], weights[0][:n//2,:])
    output1 = np.dot(output[n//2:], weights[0][n//2:,:])
    output = output0 + output1 # + output0 * output1
    output += weights[1]
    output -= output.max()
    output = np.exp(output)
    output /= output.sum()
    return output

In [35]:
def lpadd(x, maxlend=maxlend, eos=eos):
    """left (pre) pad a description to maxlend and then add eos.
    The eos is the input to predicting the first word in the headline
    """
    assert maxlend >= 0
    if maxlend == 0:
        return [eos]
    n = len(x)
    if n > maxlend:
        x = x[-maxlend:]
        n = maxlend
    return [empty]*(maxlend-n) + x + [eos]

In [36]:
samples = [lpadd([3]*26)]
# pad from right (post) so the first maxlend will be description followed by headline
data = sequence.pad_sequences(samples, maxlen=maxlen, value=empty, padding='post', truncating='post')

In [37]:
np.all(data[:,maxlend] == eos)

True

In [38]:
data.shape,list(map(len, samples))

((1, 75), [51])

In [40]:
probs = model.predict(data, verbose=0, batch_size=1)
probs.shape

(1, 25, 944)

In [83]:
def beamsearch(predict, start=[empty]*maxlend + [eos], avoid=None, avoid_score=1,
               k=1, maxsample=maxlen, use_unk=True, oov=vocab_size-1, empty=empty, eos=eos, temperature=1.0):
    """return k samples (beams) and their NLL scores, each sample is a sequence of labels,
    all samples starts with an `empty` label and end with `eos` or truncated to length of `maxsample`.
    You need to supply `predict` which returns the label probability of each sample.
    `use_unk` allow usage of `oov` (out-of-vocabulary) label in samples
    """
    def sample(energy, n, temperature=temperature):
        """sample at most n different elements according to their energy"""
        n = min(n,len(energy))
        prb = np.exp(-np.array(energy) / temperature )
        res = []
        for i in range(n):
            z = np.sum(prb)
            r = np.argmax(np.random.multinomial(1, prb/z, 1))
            res.append(r)
            prb[r] = 0. # make sure we select each element only once
        return res

    dead_samples = []
    dead_scores = []
    live_samples = [list(start)]
    live_scores = [0]

    while live_samples:
        # for every possible live sample calc prob for every possible label 
        probs = predict(live_samples, empty=empty)
        print(probs)
        print(vocab_size)
        assert vocab_size == probs.shape[1]

        # total score for every sample is sum of -log of word prb
        cand_scores = np.array(live_scores)[:,None] - np.log(probs)
        cand_scores[:,empty] = 1e20
        if not use_unk and oov is not None:
            cand_scores[:,oov] = 1e20
        if avoid:
            for a in avoid:
                for i, s in enumerate(live_samples):
                    n = len(s) - len(start)
                    if n < len(a):
                        # at this point live_sample is before the new word,
                        # which should be avoided, is added
                        cand_scores[i,a[n]] += avoid_score
        live_scores = list(cand_scores.flatten())
        

        # find the best (lowest) scores we have from all possible dead samples and
        # all live samples and all possible new words added
        scores = dead_scores + live_scores
        ranks = sample(scores, k)
        n = len(dead_scores)
        dead_scores = [dead_scores[r] for r in ranks if r < n]
        dead_samples = [dead_samples[r] for r in ranks if r < n]
        
        live_scores = [live_scores[r-n] for r in ranks if r >= n]
        live_samples = [live_samples[(r-n)//vocab_size]+[(r-n)%vocab_size] for r in ranks if r >= n]

        # live samples that should be dead are...
        # even if len(live_samples) == maxsample we dont want it dead because we want one
        # last prediction out of it to reach a headline of maxlenh
        def is_zombie(s):
            return s[-1] == eos or len(s) > maxsample
        
        # add zombies to the dead
        dead_scores += [c for s, c in zip(live_samples, live_scores) if is_zombie(s)]
        dead_samples += [s for s in live_samples if is_zombie(s)]
        
        # remove zombies from the living 
        live_scores = [c for s, c in zip(live_samples, live_scores) if not is_zombie(s)]
        live_samples = [s for s in live_samples if not is_zombie(s)]

    return dead_samples, dead_scores


In [84]:
def keras_rnn_predict(samples, empty=empty, model=model, maxlen=maxlen):
    """for every sample, calculate probability for every possible label
    you need to supply your RNN model and maxlen - the length of sequences it can handle
    """
    sample_lengths = list(map(len, samples))
    assert all(l > maxlend for l in sample_lengths)
    assert all(l[maxlend] == eos for l in samples)
    # pad from right (post) so the first maxlend will be description followed by headline
    data = sequence.pad_sequences(samples, maxlen=maxlen, value=empty, padding='post', truncating='post')
    probs = model.predict(data, verbose=0, batch_size=batch_size)
    return np.array([output2probs(prob[sample_length-maxlend-1]) for prob, sample_length in zip(probs, sample_lengths)])

In [85]:
def vocab_fold(xs):
    """convert list of word indexes that may contain words outside vocab_size to words inside.
    If a word is outside, try first to use glove_idx2idx to find a similar word inside.
    If none exist then replace all accurancies of the same unknown word with <0>, <1>, ...
    """
    xs = [x if x < vocab_size-nb_unknown_words else glove_idx2idx.get(x,x) for x in xs]
    # the more popular word is <0> and so on
    outside = sorted([x for x in xs if x >= vocab_size-nb_unknown_words])
    # if there are more than nb_unknown_words oov words then put them all in nb_unknown_words-1
    outside = dict((x,vocab_size-1-min(i, nb_unknown_words-1)) for i, x in enumerate(outside))
    xs = [outside.get(x,x) for x in xs]
    return xs

In [86]:
def vocab_unfold(desc,xs):
    # assume desc is the unfolded version of the start of xs
    unfold = {}
    for i, unfold_idx in enumerate(desc):
        fold_idx = xs[i]
        if fold_idx >= vocab_size-nb_unknown_words:
            unfold[fold_idx] = unfold_idx
    return [unfold.get(x,x) for x in xs]


In [110]:
import sys
import Levenshtein

def gensamples(X=None, X_test=None, Y_test=None, avoid=None, avoid_score=1, skips=2, k=10, batch_size=batch_size, short=True, temperature=1., use_unk=True):
    if X is None or isinstance(X,int):
        if X is None:
            i = random.randint(0,len(X_test)-1)
        else:
            i = X
        print('HEAD %d:'%i,' '.join(idx2word[w] for w in Y_test[i]))
        print('DESC:',' '.join(idx2word[w] for w in X_test[i]))
        sys.stdout.flush()
        x = X_test[i]
    else:
        x = [word2idx[w.rstrip('^')] for w in X.split()]
        
    if avoid:
        # avoid is a list of avoids. Each avoid is a string or list of word indeicies
        if isinstance(avoid,str) or isinstance(avoid[0], int):
            avoid = [avoid]
        avoid = [a.split() if isinstance(a,str) else a for a in avoid]
        avoid = [vocab_fold([w if isinstance(w,int) else word2idx[w] for w in a])
                 for a in avoid]

    print('HEADS:')
    samples = []
    if maxlend == 0:
        skips = [0]
    else:
        skips = range(min(maxlend,len(x)), max(maxlend,len(x)), abs(maxlend - len(x)) // skips + 1)
    for s in skips:
        start = lpadd(x[:s])
        fold_start = vocab_fold(start)
        sample, score = beamsearch(predict=keras_rnn_predict, start=fold_start, avoid=avoid, avoid_score=avoid_score,
                                   k=k, temperature=temperature, use_unk=use_unk)
        assert all(s[maxlend] == eos for s in sample)
        samples += [(s,start,scr) for s,scr in zip(sample,score)]

    samples.sort(key=lambda x: x[-1])
    codes = []
    for sample, start, score in samples:
        code = ''
        words = []
        sample = vocab_unfold(start, sample)[len(start):]
        for w in sample:
            if w == eos:
                break
            words.append(idx2word[w])
            code += chr(w//(256*256)) + chr((w//256)%256) + chr(w%256)
        if short:
            distance = min([100] + [-Levenshtein.jaro(code,c) for c in codes])
            if distance > -0.6:
                print(score, ' '.join(words))
        #         print '%s (%.2f) %f'%(' '.join(words), score, distance)
        else:
                print(score, ' '.join(words))
        codes.append(code)
    return samples


In [111]:
seed = 8
random.seed(seed)
np.random.seed(seed)


In [112]:
X = "* Billy Joel is looking for a buyer in Sagaponack^ . Now that he and wife Katie Lee Joel are splitting up , the singer is planning to sell the two oceanfront^ properties he bought for her in 2007 . The four-bedroom mansion ( No . 1 ) and smaller beach bungalow^ ( No . 2 ) will be listed with Corcoran 's Biana^ Stepanian^ for a combined $ 35 million . * Richard Bressler^ , the former CFO of Viacom and now a managing"
Y = "Billy Joel Lists in Sagaponack^"

In [113]:
samples = gensamples(X=X, skips=2, batch_size=batch_size, k=10, temperature=1.)

HEADS:
[[1.31904713e-07 8.21081267e-06 3.78499652e-04 ... 1.36085665e-02
  3.69094610e-02 9.16327462e-02]]
40000
[[4.2552461e-08 4.0826765e-03 7.0174425e-03 ... 1.7625712e-02
  4.6766154e-02 9.9477686e-02]
 [9.6333117e-08 7.0102827e-04 1.2776875e-02 ... 1.7636022e-02
  4.6443574e-02 1.0856983e-01]
 [4.8351851e-08 2.8615820e-03 6.9113090e-03 ... 1.6355164e-02
  4.3106023e-02 9.5353350e-02]
 ...
 [5.0410907e-08 2.4577379e-03 7.0752534e-03 ... 1.6121289e-02
  4.2652965e-02 9.5924109e-02]
 [4.4278263e-08 3.6721423e-03 6.9771591e-03 ... 1.7289892e-02
  4.5713495e-02 9.8554403e-02]
 [4.6345431e-08 3.2028183e-03 6.9371131e-03 ... 1.6805723e-02
  4.4428974e-02 9.7169459e-02]]
40000
[[5.41271525e-08 1.22895623e-02 8.54687765e-03 ... 1.75542776e-02
  4.06865925e-02 7.14391395e-02]
 [7.78551836e-08 1.10396510e-02 9.43315960e-03 ... 1.46290455e-02
  3.28875706e-02 5.30686863e-02]
 [5.27971338e-08 1.34733282e-02 8.47573206e-03 ... 1.77374091e-02
  4.10676673e-02 7.14330301e-02]
 ...
 [6.77835175e-0

[[2.8307712e-08 1.4862584e-01 9.4865570e-03 ... 2.8646905e-02
  4.5427982e-02 5.5233914e-02]
 [1.3820147e-08 3.4337357e-01 6.5974137e-03 ... 2.5266267e-02
  3.8459845e-02 4.2574134e-02]
 [2.1435476e-08 2.2582793e-01 7.9546338e-03 ... 2.7606601e-02
  4.2720623e-02 4.9594309e-02]
 ...
 [2.0039666e-08 2.4192555e-01 7.7616381e-03 ... 2.7448406e-02
  4.2405013e-02 4.8926681e-02]
 [1.4481310e-08 3.4483108e-01 6.6000372e-03 ... 2.5199166e-02
  3.8302381e-02 4.2162344e-02]
 [2.2611735e-08 2.1027261e-01 8.2404269e-03 ... 2.7940085e-02
  4.3434955e-02 5.0884347e-02]]
40000
[[9.2925518e-08 1.5491252e-02 4.0698264e-02 ... 3.0685948e-02
  5.3545784e-02 6.8470329e-02]
 [4.1611468e-08 1.2846227e-01 1.4581386e-02 ... 2.7903540e-02
  4.1766811e-02 4.8191801e-02]
 [9.1326505e-08 1.1360171e-02 4.7956795e-02 ... 3.2982279e-02
  6.0846154e-02 7.9394981e-02]
 ...
 [1.3568033e-08 4.3226981e-01 6.5036817e-03 ... 2.1123225e-02
  2.9240036e-02 2.9556453e-02]
 [1.9581577e-08 3.1153101e-01 7.6697152e-03 ... 2.453

[[1.8193024e-08 1.4235686e-01 6.9870530e-03 ... 2.6190724e-02
  5.4734357e-02 6.9183059e-02]
 [1.7584872e-08 1.3652350e-01 7.0551927e-03 ... 2.7131705e-02
  5.6708507e-02 7.3082030e-02]
 [4.0053195e-08 1.6580293e-02 1.0817546e-02 ... 2.1487413e-02
  4.8725341e-02 7.4180096e-02]
 ...
 [1.2001182e-08 1.9534792e-01 7.5900531e-03 ... 1.9282676e-02
  4.2432349e-02 4.9458362e-02]
 [1.3215789e-08 1.6206193e-01 5.5390298e-03 ... 2.3052026e-02
  4.7149803e-02 5.9495490e-02]
 [1.1111147e-08 1.9086428e-01 5.4704626e-03 ... 2.3101825e-02
  4.7303379e-02 5.9079930e-02]]
40000
[[4.5327560e-08 3.7122481e-02 1.2344778e-02 ... 2.1431657e-02
  4.3873321e-02 5.8636252e-02]
 [2.4438156e-08 1.9088802e-01 8.2993619e-03 ... 2.4498390e-02
  4.7329925e-02 5.4332461e-02]
 [1.1970144e-08 2.8152606e-01 5.9703258e-03 ... 2.0479327e-02
  3.8271770e-02 4.2874165e-02]
 ...
 [1.6387485e-08 2.2662716e-01 6.3229948e-03 ... 2.0699285e-02
  3.8645159e-02 4.3687403e-02]
 [1.2604504e-08 2.7944261e-01 6.0372087e-03 ... 2.064

In [114]:
X = "18 Cake GIFs That 'll Make You Moist"
Y = "Is it 350degF^ in here or is it just me ?"
samples = gensamples(X, skips=2, batch_size=batch_size, k=10, temperature=1.)


HEADS:
[[6.4165448e-07 1.6674654e-04 1.0682308e-03 ... 5.9778686e-03
  1.4859438e-02 4.2210981e-02]]
40000
[[3.90444939e-07 1.21612875e-02 3.16354726e-03 ... 2.60647619e-03
  5.14106173e-03 9.83450282e-03]
 [3.88807592e-07 1.23917898e-02 3.13178264e-03 ... 2.58147647e-03
  5.07926941e-03 9.66617279e-03]
 [3.96748305e-07 1.11151719e-02 3.05838999e-03 ... 2.50119530e-03
  4.90070228e-03 9.41967964e-03]
 ...
 [4.00394242e-07 1.10243298e-02 3.04598687e-03 ... 2.49085692e-03
  4.87815542e-03 9.37212072e-03]
 [3.84918394e-07 1.25281001e-02 3.14135500e-03 ... 2.59002578e-03
  5.09466464e-03 9.69569292e-03]
 [5.25477276e-07 3.72617738e-03 2.99098995e-03 ... 1.94890879e-03
  3.90219339e-03 8.28481186e-03]]
40000
[[3.4138483e-07 3.4002751e-02 4.6220105e-03 ... 2.6499873e-03
  4.7221784e-03 7.6346225e-03]
 [5.3008324e-07 1.1366657e-02 4.1170642e-03 ... 1.9957991e-03
  3.5976134e-03 6.2198266e-03]
 [4.4473447e-07 2.1040624e-02 3.9486364e-03 ... 2.1932831e-03
  3.8588378e-03 6.2926081e-03]
 ...
 [3

[[2.4528666e-07 2.6206905e-01 3.1825479e-03 ... 3.2174182e-03
  3.8725710e-03 3.9310767e-03]
 [2.2805058e-07 2.0587121e-01 2.6313656e-03 ... 3.3872025e-03
  4.3095760e-03 4.7112582e-03]
 [1.1335289e-06 5.3398600e-03 1.6841860e-02 ... 3.8077091e-03
  6.1063864e-03 8.1939586e-03]
 ...
 [5.9206690e-07 1.9816814e-02 3.9470615e-03 ... 2.6989316e-03
  3.8180666e-03 5.0821728e-03]
 [2.3259396e-07 2.0531419e-01 2.6225771e-03 ... 3.3682955e-03
  4.2813262e-03 4.6688532e-03]
 [6.6080628e-07 1.4649253e-02 4.4395621e-03 ... 2.6871997e-03
  3.8862359e-03 5.2696401e-03]]
40000
[[2.8534521e-07 1.4421494e-01 2.6081845e-03 ... 3.3194046e-03
  4.2843893e-03 4.8400490e-03]
 [4.8711507e-07 8.6195715e-02 4.1839411e-03 ... 3.0693123e-03
  3.8365871e-03 4.2660874e-03]
 [3.0482505e-07 1.3612589e-01 2.9792027e-03 ... 3.3595844e-03
  4.2785071e-03 4.8278482e-03]
 ...
 [2.0875262e-07 2.3737644e-01 2.8638032e-03 ... 3.5168552e-03
  4.4386787e-03 4.8032170e-03]
 [2.0592891e-07 2.4394619e-01 2.9013674e-03 ... 3.506

[[2.56260620e-07 1.45080715e-01 2.44057109e-03 ... 3.28333536e-03
  4.76753525e-03 5.70872892e-03]
 [3.94089625e-07 1.26063779e-01 2.93846149e-03 ... 2.81300349e-03
  3.83167877e-03 4.27344814e-03]
 [7.98784470e-07 5.86404186e-03 5.35880588e-03 ... 2.58444133e-03
  4.48756525e-03 6.81907404e-03]
 ...
 [6.29694512e-07 3.08370925e-02 4.33363253e-03 ... 2.55241874e-03
  3.75072332e-03 4.75794962e-03]
 [4.28158046e-07 1.05312325e-01 3.08165303e-03 ... 2.79509719e-03
  3.83857847e-03 4.36057663e-03]
 [3.21244585e-07 9.57176536e-02 2.31562532e-03 ... 2.87689292e-03
  4.13175719e-03 4.97207837e-03]]
40000
[[2.3064543e-07 1.8957493e-01 2.6424511e-03 ... 3.4350031e-03
  4.7873515e-03 5.5351360e-03]
 [3.1208469e-07 1.9827327e-01 3.0540556e-03 ... 3.0977819e-03
  4.0345676e-03 4.3348079e-03]
 [2.4632189e-07 1.8426876e-01 2.6530928e-03 ... 3.4253749e-03
  4.7756345e-03 5.5128895e-03]
 ...
 [2.7056618e-07 1.4270122e-01 2.5967867e-03 ... 3.3199331e-03
  4.6506077e-03 5.4973853e-03]
 [3.9094635e-07 1

In [115]:
X = "President Barack Obama 's re-election campaign is fundraising off of comments on Obama 's birth certificate by Mitt Romney 's son Matt ."

In [116]:
gensamples(X, skips=2, batch_size=batch_size, k=10, temperature=1, use_unk=True, short=False);

HEADS:
[[1.3904784e-07 1.0756031e-05 3.4826784e-04 ... 8.0823796e-03
  3.4020886e-02 1.0930793e-01]]
40000
[[7.95786406e-08 6.83127204e-04 1.17748985e-02 ... 8.94693937e-03
  3.37437503e-02 1.12477720e-01]
 [6.50957688e-08 1.42202992e-03 8.19212478e-03 ... 8.53776745e-03
  3.14509571e-02 9.92092416e-02]
 [8.35186924e-08 4.63114149e-04 2.03659460e-02 ... 1.12329731e-02
  4.33352552e-02 1.45629480e-01]
 ...
 [4.17816537e-08 3.46750324e-03 7.16519170e-03 ... 9.41496715e-03
  3.46978419e-02 1.00834057e-01]
 [7.30455767e-08 8.65066831e-04 1.02835903e-02 ... 8.74020811e-03
  3.27540785e-02 1.07889034e-01]
 [8.17117609e-08 5.77614410e-04 1.42483544e-02 ... 9.49240942e-03
  3.60742323e-02 1.21412091e-01]]
40000
[[8.27124893e-08 6.58008503e-03 8.53559282e-03 ... 6.66423468e-03
  2.06413474e-02 4.54830974e-02]
 [1.41017040e-07 2.30491743e-03 9.99842025e-03 ... 5.64442901e-03
  1.78925544e-02 4.23786938e-02]
 [1.05117792e-07 4.26064897e-03 8.77137948e-03 ... 6.12709811e-03
  1.90394372e-02 4.3283

[[6.9857464e-08 7.5989487e-03 8.9902794e-03 ... 6.9595282e-03
  2.1768630e-02 4.7930200e-02]
 [1.3586059e-07 1.9664695e-03 1.2141375e-02 ... 6.1250040e-03
  1.9979099e-02 4.8820160e-02]
 [7.5955256e-08 6.4816456e-03 9.0709021e-03 ... 6.6756210e-03
  2.0749595e-02 4.6036590e-02]
 ...
 [1.4461585e-07 3.4249612e-04 4.4760965e-02 ... 8.7415446e-03
  3.3840541e-02 9.5561109e-02]
 [1.5565206e-07 7.1297685e-04 2.3017218e-02 ... 6.9145490e-03
  2.4609424e-02 6.6272236e-02]
 [1.2285241e-07 4.7440096e-03 9.8626930e-03 ... 7.0053232e-03
  2.2263167e-02 5.0484311e-02]]
40000
[[1.02602861e-07 7.18215888e-04 4.06881124e-02 ... 7.88896345e-03
  3.00478898e-02 7.63314441e-02]
 [1.01393944e-07 7.28271611e-04 4.08715121e-02 ... 7.92511553e-03
  3.02040335e-02 7.68167600e-02]
 [3.00735863e-08 1.58799198e-02 1.05603402e-02 ... 7.55149079e-03
  2.43044868e-02 4.91022430e-02]
 ...
 [3.19331939e-08 1.49886943e-02 1.06271831e-02 ... 7.46545894e-03
  2.39547659e-02 4.84767482e-02]
 [3.02986756e-08 1.57833360e-

In [117]:
X = "What have you been listening to this year ? If you want to find out using cold , hard evidence , then Spotify 's new Year in Music tool will tell you ."
Y = "Spotify Will Make You Smarter for Your App"


In [118]:
samples = gensamples(X, skips=2, batch_size=batch_size, k=10, temperature=1)

HEADS:
[[1.81063783e-07 2.53055568e-05 5.04992146e-04 ... 1.51469335e-02
  4.30171713e-02 1.17688522e-01]]
40000
[[8.52222186e-08 1.49350800e-03 1.64321549e-02 ... 1.73506252e-02
  5.03322594e-02 1.34962022e-01]
 [4.34987371e-08 1.07661933e-02 8.11931305e-03 ... 1.82052981e-02
  5.27953878e-02 1.19102538e-01]
 [3.07918135e-08 1.35436850e-02 8.25380906e-03 ... 1.86277907e-02
  5.30725680e-02 1.20296210e-01]
 ...
 [4.86176042e-08 4.69500897e-03 9.82294697e-03 ... 1.65204927e-02
  4.80262339e-02 1.21654145e-01]
 [3.56029730e-08 1.05841039e-02 8.46204814e-03 ... 1.80631578e-02
  5.14267795e-02 1.19593710e-01]
 [4.47076047e-08 6.39291946e-03 8.87771696e-03 ... 1.66486055e-02
  4.77352329e-02 1.16140693e-01]]
40000
[[4.1448434e-08 3.5560023e-02 1.2369523e-02 ... 1.5655030e-02
  3.8869236e-02 6.7635916e-02]
 [4.5871701e-08 3.1771451e-02 1.2251535e-02 ... 1.5243645e-02
  3.7730630e-02 6.5867111e-02]
 [6.5362386e-08 2.7443576e-03 2.2685120e-02 ... 1.6526118e-02
  4.6898451e-02 1.1600130e-01]
 .

[[1.1312005e-08 3.4331694e-01 6.6417465e-03 ... 1.9754771e-02
  3.4704745e-02 4.4300839e-02]
 [1.8732306e-08 2.7097139e-01 8.4621683e-03 ... 2.2059038e-02
  3.9451096e-02 5.0549630e-02]]
40000
[[2.2152946e-08 3.3113214e-01 1.0103261e-02 ... 2.0249300e-02
  3.3748269e-02 3.9763357e-02]]
40000
[[1.81063783e-07 2.53055568e-05 5.04992146e-04 ... 1.51469335e-02
  4.30171713e-02 1.17688522e-01]]
40000
[[4.8878245e-08 4.7498178e-03 9.6199568e-03 ... 1.6358843e-02
  4.7465418e-02 1.1965624e-01]
 [8.5222219e-08 1.4935080e-03 1.6432155e-02 ... 1.7350625e-02
  5.0332259e-02 1.3496202e-01]
 [4.0797794e-08 8.0545163e-03 8.6414618e-03 ... 1.7229984e-02
  4.9155165e-02 1.1710809e-01]
 ...
 [4.6100087e-08 5.6065097e-03 9.3642110e-03 ... 1.6716655e-02
  4.8244853e-02 1.1984112e-01]
 [5.4528240e-08 3.5369755e-03 1.0258050e-02 ... 1.5825100e-02
  4.6299424e-02 1.1933611e-01]
 [4.3498737e-08 1.0766193e-02 8.1193130e-03 ... 1.8205298e-02
  5.2795388e-02 1.1910254e-01]]
40000
[[5.53492754e-08 2.41810512e-02

[[4.4340837e-08 5.9098944e-02 1.5517384e-02 ... 2.0506479e-02
  3.7265524e-02 5.2931510e-02]]
40000
7.9298484325408936 <0>^ <0>^


In [119]:
headline = samples[0][0][len(samples[0][1]):]
print(headline)

[39999, 39999, 1]


In [120]:
' '.join(idx2word[w] for w in headline)

'<0>^ <0>^ ~'

In [121]:
avoid = headline
print (avoid)

[39999, 39999, 1]


In [123]:

len(samples)


20